# Calculate Scanpaths Experience Level with XY Coordinates


This notebook will calculate scanpaths for every experience group. I will take a look at the mean for every scene and image for every experience level.


In [ ]:
from matplotlib import image 
from matplotlib import pyplot as plt 
import pandas as pd
import numpy as np

pad = "../../data/csv/"
image_pad =  "../../data/Images/CrimeScenes/"

df = pd.read_csv(pad + 'fix_evidence.csv')

The function calculate_distance will calculate the distance based on an X and Y column and X1 and Y1 colomn.
This is the mean for every experience level over all scenes and images:


In [2]:
def calculate_distance(row):
    if pd.isnull(row['X']) or pd.isnull(row['X1']):
        return np.nan
    elif row['FixNr'] == 1:
        return 0
    else:
        return round(np.sqrt((row['X1'] - row['X'])**2 + (row['Y1'] - row['Y'])**2), 2)

This code wil make an X1 and Y1 colomn with the first row a zero. Next it will take the previous value of the X column and put it in the X1 colomn. I also made a new colomn called distance that calculates the distance from these values.


In [3]:
# Put X and Y on zero for first row because you do not have a previous row
df.at[0, 'X1'] = 0
df.at[0, 'Y1'] = 0

# Fill subsequent rows with the previous row's values for X1 and Y1 if they haven't been reset
# for i in range(1, len(df)):
#     if df.at[i, 'X1'] == 0:
#         df.at[i, 'X1'] = df.at[i-1, 'X']
#     if df.at[i, 'Y1'] == 0:
#         df.at[i, 'Y1'] = df.at[i-1, 'Y']

#Fill subsequent rows with the previous row's values for X1 and Y1
for i in range(1, len(df)):
    if df.at[i, 'FixNr'] != 1:
        df.at[i, 'X1'] = df.at[i-1, 'X']
        df.at[i, 'Y1'] = df.at[i-1, 'Y']
    else:
        df.at[i, 'X1'] = 0
        df.at[i, 'Y1'] = 0
    
        
# Iterate through the rows to set X1 and Y1 to 0 based on Participant, Image, and Scene changes
# for i in range(1, len(df)):
#     if (df.at[i, 'Participant'] != df.at[i-1, 'Participant']) or \
#        (df.at[i, 'Image'] != df.at[i-1, 'Image']) or \
#        (df.at[i, 'Scene'] != df.at[i-1, 'Scene']):
#         df.at[i, 'X1'] = 0
#         df.at[i, 'Y1'] = 0
    
df['Distance'] = df.apply(lambda row: calculate_distance(row), axis=1)

In [4]:
df.to_csv('distance.csv')

In [5]:
filtered_df = df[df['experience'] == 'Control']
df_grouped = filtered_df.groupby(['Participant', 'Scene', 'Image'])['Distance'].sum().reset_index()
avg = df_grouped['Distance'].mean()

In [6]:
filtered_df

,Unnamed: 0,Image,Scene,ROI,Participant,FixDur,FixNr,Start,End,X,Y,Name,NextROI,experience,Evidence,Movable,ExitEntry,X1,Y1,Distance
0,0,1,1,0,2.0,103.0,1.0,7.0,109.0,510.9,381.0,Back_wall,Bottle_on_table,Control,No,No,No,0.0,0.0,0.00
1,1,1,1,22,2.0,77.0,2.0,236.0,312.0,450.8,307.0,Bottle_on_table,Floor,Control,Yes,Yes,No,510.9,381.0,95.33
2,2,1,1,9,2.0,300.0,3.0,347.0,646.0,414.5,447.9,Floor,Body,Control,No,No,No,450.8,307.0,145.50
3,3,1,1,8,2.0,165.0,4.0,687.0,851.0,468.6,619.1,Body,Elsewhere,Control,Yes,Yes,No,414.5,447.9,179.54
4,4,1,1,0,2.0,461.0,5.0,876.0,1336.0,465.4,687.9,Body,Elsewhere,Control,Yes,Yes,No,468.6,619.1,68.87
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30187,30187,2,3,7,9008.0,243.0,71.0,25535.0,25777.0,307.6,524.2,Table,NaN,Control,No,No,No,618.6,465.5,316.49
30188,30188,2,3,13,9008.0,274.0,72.0,25829.0,26102.0,530.8,385.8,Right_plate,NaN,Control,Yes,Yes,No,307.6,524.2,262.63
30189,30189,2,3,14,9008.0,374.0,73.0,26136.0,26509.0,623.9,466.2,Right_fork,NaN,Control,Yes,Yes,No,530.8,385.8,123.01
30190,30190,2,3,7,9008.0,170.0,74.0,26578.0,26747.0,305.1,542.6,Table,NaN,Control,No,No,No,623.9,466.2,327.83


In [7]:
def avg_exp():
    exps = df['experience'].unique().tolist()
    avgs = {}
    for exp in exps:
        filtered_df = df[df['experience'] == exp]
        df_grouped = filtered_df.groupby(['Participant', 'Scene', 'Image'])['Distance'].sum().reset_index()
        avg = df_grouped['Distance'].mean()
        avgs[exp] = avg
    return avgs

In [8]:
result = avg_exp()
for exp in result:
    print(f'{exp} has an average scanpath of: {int(result[exp])}')

Control has an average scanpath of: 15965
FirstYear has an average scanpath of: 13926
ThirdYear has an average scanpath of: 17949
CSI has an average scanpath of: 16681


This is the mean for every expierence level for the scenes combined:


In [9]:
def avg_scene_exp():
    exps = df['experience'].unique().tolist()
    scenes = df['Scene'].unique().tolist()
    avgs={}
    for exp in exps:
        filtered_df = df[df['experience'] == exp]
        avgs[exp] = {}
        for scene in scenes:
            df_scene = filtered_df[filtered_df['Scene'] == scene]
            df_grouped = df_scene.groupby(['Participant', 'Image'])['Distance'].sum().reset_index()
            avg = df_grouped['Distance'].mean()
            avgs[exp][scene] = avg
    return avgs

In [10]:
result = avg_scene_exp()
for exp in result:
    for scene in result[exp]:
        print(f'{exp} has an average scanpath in {scene} of: {round((result[exp][scene]), 2)}')


Control has an average scanpath in 1 of: 17413.5
Control has an average scanpath in 2 of: 17136.45
Control has an average scanpath in 3 of: 13346.21
FirstYear has an average scanpath in 1 of: 17430.4
FirstYear has an average scanpath in 2 of: 13453.04
FirstYear has an average scanpath in 3 of: 11283.99
ThirdYear has an average scanpath in 1 of: 19337.61
ThirdYear has an average scanpath in 2 of: 18537.07
ThirdYear has an average scanpath in 3 of: 15974.43
CSI has an average scanpath in 1 of: 18811.8
CSI has an average scanpath in 2 of: 16662.1
CSI has an average scanpath in 3 of: 14570.17


This is the mean for every expierence level for the scenes and images:


In [11]:
def avg_scene_image_exp():
    exps = df['experience'].unique().tolist()
    scenes = df['Scene'].unique().tolist()
    images = df['Image'].unique().tolist()
    avgs={}
    for exp in exps:
        filtered_df = df[df['experience'] == exp]
        avgs[exp] = {}
        for scene in scenes:
            df_scene = filtered_df[filtered_df['Scene'] == scene]
            avgs[exp][scene] = {}
            for image in images:
                df_image = filtered_df[filtered_df['Image'] == image]
                df_grouped = df_image.groupby('Participant')['Distance'].sum().reset_index()
                avg = df_grouped['Distance'].mean()
                avgs[exp][scene][image] = avg
    return avgs

In [12]:
result = avg_scene_image_exp()
for exp in result:
    for scene in result[exp]:
        for image in result[exp][scene]:
            print(f'{exp} has an average scanpath in {scene} {image} of: {int(result[exp][scene][image])}')

Control has an average scanpath in 1 1 of: 51389
Control has an average scanpath in 1 2 of: 44402
Control has an average scanpath in 2 1 of: 51389
Control has an average scanpath in 2 2 of: 44402
Control has an average scanpath in 3 1 of: 51389
Control has an average scanpath in 3 2 of: 44402
FirstYear has an average scanpath in 1 1 of: 40470
FirstYear has an average scanpath in 1 2 of: 39991
FirstYear has an average scanpath in 2 1 of: 40470
FirstYear has an average scanpath in 2 2 of: 39991
FirstYear has an average scanpath in 3 1 of: 40470
FirstYear has an average scanpath in 3 2 of: 39991
ThirdYear has an average scanpath in 1 1 of: 56556
ThirdYear has an average scanpath in 1 2 of: 51141
ThirdYear has an average scanpath in 2 1 of: 56556
ThirdYear has an average scanpath in 2 2 of: 51141
ThirdYear has an average scanpath in 3 1 of: 56556
ThirdYear has an average scanpath in 3 2 of: 51141
CSI has an average scanpath in 1 1 of: 55056
CSI has an average scanpath in 1 2 of: 45031
CSI 